# Split tifs

## Inputs
This program takes the DEM_Slope.tif files and splits them into 1024x768 tifs and jpgs for use in training the model and running predictions.

DEM_Slope.tif files represent a PA State Game Land area and have an area number. The variable area_str represents the number in the program.

Also, each tif is either a PAN or PAS depending on the coordinate reference system (CRS) used for it. PAN is CRS 32128. PAS is 32129. Some Game Land areas have both PAN and PAS CRS .tifs, such as area 58. The variable panorpas represents whether the area tif file being processed is a PAN or a PAS.

To classify files for model training purposes, the program uses a shapefile of known charcoal hearths.
/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp

## Function
The program takes a large tif and then divides it into rectangles of standard size 1024x768 pixels. 

## Output
The program stores .tifs in:
/storage/images/charcoal_hearth_hill/images/split_tifs/

.jpgs in:
/storage/images/charcoal_hearth_hill/images/jpgs/

The program creates a .shp file to show the layout of the split tifs. These .shp files are in: 
/storage/images/charcoal_hearth_hill/polys/
Files are named: area_str + panorpas "_tile_boundary.shp"

In [64]:
# %%pycodestyle
# Run this to make sure all of the directories exist
data_directory = "/home/student/charcoal_hearth_hill/"
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
print(data_directory)
data_sub_directory = os.path.join(data_directory, "images")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_sub_directory = os.path.join(data_sub_directory, "jpgs")
if not os.path.exists(data_sub_sub_directory):
    os.makedirs(data_sub_sub_directory)
print(data_sub_sub_directory)
data_sub_sub_directory = os.path.join(data_sub_directory, "split_tifs")
if not os.path.exists(data_sub_sub_directory):
    os.makedirs(data_sub_sub_directory)
print(data_sub_sub_directory)
data_sub_directory = os.path.join(data_directory, "polys")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "area_hearths")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "annots")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "images_training")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)

/home/student/charcoal_hearth_hill/
/home/student/charcoal_hearth_hill/images
/home/student/charcoal_hearth_hill/images/jpgs
/home/student/charcoal_hearth_hill/images/split_tifs
/home/student/charcoal_hearth_hill/polys
/home/student/charcoal_hearth_hill/area_hearths
/home/student/charcoal_hearth_hill/annots
/home/student/charcoal_hearth_hill/images_training


In [1]:
import geopandas as gpd
#https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
from osgeo import osr, gdal

def get_poly_with_lat_long_from_geotif(geotif_fp):
    # Import necessary modules


    # get the existing coordinate system
    ds = gdal.Open(geotif_fp)

    old_cs= osr.SpatialReference()

    old_cs.ImportFromWkt(ds.GetProjectionRef())

    # create the new coordinate system
    nad83_wkt = """
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]]"""

    new_cs = osr.SpatialReference()

    new_cs .ImportFromWkt(nad83_wkt)

    # create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    #get the point to transform, pixel (0,0) in this case
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    #get the coordinates in lat long
    latlong1 = transform.TransformPoint(minx,miny)
    #print(latlong1)
    latlong2 = transform.TransformPoint(minx,maxy)
    #print(latlong2)
    latlong3 = transform.TransformPoint(maxx,maxy)
    #print(latlong3)
    latlong4 = transform.TransformPoint(maxx,miny)
    #print(latlong4)
    
    # Create an empty geopandas GeoDataFrame
    polydata = gpd.GeoDataFrame()
    polydata['geometry'] = None

    # Create a Polygon
    #print(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])
    coords = [(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])]
    poly = Polygon(coords)

    return(poly)

import numpy as np

def tif_to_jpg(tif_file,output_file_name, output_folder):
    #print(tif_file,output_file_name, output_folder)
    ds = gdal.Open(tif_file)
    #geoTrans = srcImage.GetGeoTransform()

    band = ds.GetRasterBand(1)
    width = ds.RasterXSize
    height = ds.RasterYSize

    data = band.ReadAsArray(0, 0, width, height)
    #convert all the bad data
    data[data==-9999.0] = 0
    max_value = numpy.max(data)
    color_multiplier = 255/(max_value-1)
    #print(color_multiplier)
    data = data*color_multiplier
    #print(data)
    data_int = np.array(data, dtype='int')
    #print(data_int)
    #clip = ds.readasarray(ds)
    data_int = data_int.astype(gdalnumeric.numpy.uint8)

    gdalnumeric.SaveArray(data_int, output_folder+output_file_name+".jpg", format="JPEG")

#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html?highlight=rasterize

# Run this
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # This generates the xml file annotation that is consumed by the MASK R-CNN process.
    # With credit to: # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        
        #Sometimes the mouse gets dragged from the bottom to the top, etc
        if(refPtMin[1]>refPtMax[1]):
            ytemp = refPtMin[1]
            refPtMin[1] = refPtMax[1]
            refPtMax[1]=ytemp
        if(refPtMin[0]>refPtMax[0]):
            xtemp = refPtMin[0]
            refPtMin[0] = refPtMax[0]
            refPtMax[0]=xtemp 
        
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()

In [58]:
geotif_crs = 32129
# Create an empty geopandas GeoDataFrame
hearth_data = gpd.GeoDataFrame(columns=["geometry"], crs=geotif_crs)
#hearth_data['geometry'] = None
#hearth_data.geometry.to_crs(geotif_crs)
hearth_data = hearth_data.to_crs(crs=geotif_crs)
print(hearth_data.geometry.crs)
hearth_data.geometry = hearth_data.geometry.to_crs(crs=4326)
print(hearth_data.geometry.crs)


epsg:32129
epsg:4326


In [74]:
import gdal
from osgeo import ogr
def get_pixel_with_geot(geoT, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    raster: string
        A GDAL-friendly raster filename
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    #g = gdal.Open(raster)
    #if g is None:
    #    raise ValueError(f"{raster:s} cannot be opened!")
    #geoT = open_raster.GetGeoTransform()
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

def annotate_tif_if_it_has_hearths(area_hearth_data, tif_poly, tif_fp, annot_file_name, geotif_crs):
    # Create an empty geopandas GeoDataFrame
    # Reworked for new CRS defintions
    hearth_data = gpd.GeoDataFrame(columns=["geometry"], crs=geotif_crs)
    hearth_data = hearth_data.to_crs(crs=geotif_crs)
    hearth_data.geometry = hearth_data.geometry.to_crs(crs=geotif_crs)
    # Create an empty geopandas GeoDataFrame
    hearth_data_poly = gpd.GeoDataFrame(columns=["geometry"], crs=geotif_crs)
    hearth_data_poly = hearth_data_poly.to_crs(crs=geotif_crs)
    hearth_data_poly.geometry = hearth_data_poly.geometry.to_crs(crs=geotif_crs)
    

    selection = area_hearth_data[0:]
    point_count=0
    
    print(tif_fp)
    src_ds=gdal.Open(tif_fp) 
    gt=src_ds.GetGeoTransform()
    
    # create a transform object to convert between coordinate systems
    #transform = osr.CoordinateTransformation(old_cs,new_cs)
    
    target = osr.SpatialReference(wkt=src_ds.GetProjection())
    source = osr.SpatialReference()
    #source.ImportFromEPSG(4326)
    #source.ImportFromEPSG(32128)
    #target.ImportFromEPSG(32128)
    source.ImportFromEPSG(geotif_crs)
    target.ImportFromEPSG(geotif_crs)
    
    transform = osr.CoordinateTransformation(source, target)

    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(tif_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            point_count=point_count+1
            #print(pt, pt.within(tif_poly))
            hearth_data = hearth_data.append(row, ignore_index=True)
            
            polygon_half_size = 0.0000895078
            polygon_half_size = 0.0001745402
            #get the coordinates in lat long
            # pp_ = point_poly - the polygon around the points
            pp_minx=pt.bounds[0]-polygon_half_size
            #print(pt.bounds[0])
            pp_maxx=pt.bounds[0]+polygon_half_size
            pp_miny=pt.bounds[1]-polygon_half_size
            pp_maxy=pt.bounds[1]+polygon_half_size
            
            pp_latlong1 = transform.TransformPoint(pp_minx,pp_miny)
            #print("latlong1",latlong1)
            print("pp_minx",pp_minx)
            pp_latlong2 = transform.TransformPoint(pp_minx,pp_maxy)
            #print(latlong2)
            pp_latlong3 = transform.TransformPoint(pp_maxx,pp_maxy)
            #print(latlong3)
            pp_latlong4 = transform.TransformPoint(pp_maxx,pp_miny)
            #print(latlong4)
            # Create a Polygon
            pp_coords = [(pp_minx, pp_miny), (pp_minx, pp_maxy), (pp_maxx, pp_maxy), (pp_maxx, pp_miny)]
            ppoly = Polygon(pp_coords)
            new_pp_row = {'id':row['id'], 'geometry':ppoly}
            #hearth_data_poly.loc[0, 'geometry'] = ppoly
            #hearth_data_poly.loc[0, 'id'] = row['id']
            hearth_data_poly = hearth_data_poly.append(new_pp_row, ignore_index=True)
    #Do we have points inside this tif?  If so, annotate and copy>0
    if(point_count)>0:

        #hearth_data_poly = hearth_data_poly.to_crs(epsg=32128)
        #hearth_data = hearth_data.to_crs(epsg=32128)
        print("geotif_crs",geotif_crs)
        print(hearth_data_poly)
        print(hearth_data_poly.crs)
        hearth_data_poly.crs = ("EPSG:" + str(geotif_crs))
        hearth_data_poly.geometry = hearth_data_poly.geometry.to_crs(crs=geotif_crs)
        hearth_data_poly.to_crs(crs=geotif_crs)
        hearth_data_poly = hearth_data_poly.to_crs(epsg=geotif_crs)
        print("-------")
        hearth_data.crs = ("EPSG:" + str(geotif_crs))
        hearth_data.geometry = hearth_data.geometry.to_crs(crs=geotif_crs)
        hearth_data.to_crs(crs=geotif_crs)
        hearth_data = hearth_data.to_crs(epsg=geotif_crs)
        

    
        #Write out shp files for diagnostics
        outfp_base = tif_fp[:-4]
        outfp = outfp_base+"_hearth_data_poly.shp"
        # Write the data into that Shapefile
        hearth_data_poly.to_file(outfp)
        outfp = outfp_base+"_hearth_data.shp"
        # Write the data into that Shapefile
        hearth_data.to_file(outfp)
        print("****")

        #Get the dimentions of column and row
        nc   = src_ds.RasterXSize
        nr   = src_ds.RasterYSize
        print("src_ds.RasterXSize", nc,nr)
        gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid
        
        #target = osr.SpatialReference(wkt=src_ds.GetProjection())
        #source = osr.SpatialReference()
        #source.ImportFromEPSG(4326)
        #transform = osr.CoordinateTransformation(source, target)
        refPts = []      
        refPt = []
        #I'm hoping the tifs will be 1024X768        
        # Make annotation file for the jpeg from polygons
        for index, row in hearth_data_poly.iterrows():
            poly = row['geometry']
            #print(row, row['id'])
            #print(poly)
            
            boundary = poly.bounds
            print("boundary = poly.bounds")
            print(boundary[0])
            print(boundary[1])
            print(boundary[2])
            print(boundary[3])

            lon = boundary[0]
            lat = boundary[1]
    
            print("lon,lat",lon, lat)
      
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p1x,p1y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p1x,p1y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p1",point.GetX(), point.GetY(),p1x,p1y)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p1x,p1y= (get_pixel_with_geot(gt,cx,cy ))
            lon = boundary[2]
            lat = boundary[3]
    
            print("lon,lat",lon, lat)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p2x,p2y= (get_pixel_with_geot(gt,cx,cy ))
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p2x,p2y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p2x,p2y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p2",point.GetX(), point.GetY(),p2x,p2y)

            #p2y is smaller than p1y so we'll exchange them to keep p1 as the minimum.
            ptemp=p1y
            p1y=p2y
            p2y=ptemp
            if(p1x<0):
                p1x=0
            if(p1y<0):
                p1y=0
            if(p2x<0):
                p2x=0
            if(p2y<0):
                p2y=0

            if(p1x>nc):
                p1x=nc
            if(p1y>nr):
                p1y=nr
            if(p2x>nc):
                p2x=nc
            if(p2y>nr):
                p2y=nr
     
        
            refPt = [(p1x, p1y)]
            refPt.append((p2x, p2y))
            refPts.append(refPt)

        object_type = 'charcoal_hearth_hill'
        
        object_annots_dir = '/home/student/charcoal_hearth_hill/annots/'
        write_annot(object_type, object_annots_dir, annot_file_name, annot_file_name+".jpg",nc,nr,1, refPts)
        
        images_training_dir = '/home/student/charcoal_hearth_hill/images_training/'
        tif_to_jpg(tif_fp,annot_file_name,images_training_dir)
    

In [77]:
# Thanks to
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os  
import numpy
#import gdal
from osgeo import gdal, osr
import math
from itertools import chain
#import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

def get_extent(dataset):

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy),
            "cols": str(cols), "rows": str(rows)
            }

def create_tiles(minx, miny, maxx, maxy, nw, nh):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(nw, 0, -1):
        for i in range(0, nh):

            ulx = minx + (width/nw) * i # 10/5 * 1
            uly = miny + (height/nh) * j # 10/5 * 1

            lrx = minx + (width/nw) * (i + 1)
            lry = miny + (height/nh) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix



def create_standard_tiles(minx, miny, maxx, maxy):
    print("standard",minx, miny, maxx, maxy)
    width = maxx - minx
    height = maxy - miny
    
    standard_width = 1024
    standard_height = 768
   
    nw = int(width/standard_width)+1
    nh = int(height/standard_height)+1
    print(nw,nh)
    #print(nw*standard_width,nh*standard_height)
    
    matrix = []

    for j in range(nh, 0, -1):
        for i in range(0, nw):

            ulx = minx + (standard_width * i) # 10/5 * 1
            uly = miny + (standard_height * j) # 10/5 * 1

            lrx = minx + (standard_width * (i + 1))
            lry = miny + (standard_height * (j - 1))
            if(lrx>maxx):
                lrx=maxx
            if(lrx<minx):
                lrx=minx
              
            if(lry>maxy):
                lry=maxy
            if(lry<miny):
                lry=miny

            if(ulx>maxx):
                ulx=maxx
            if(ulx<minx):
                ulx=minx

            if(uly>maxy):
                uly=maxy
            if(uly<miny):
                uly=miny

            #print("matrix:",ulx, uly,lrx, lry)
            matrix.append([[ulx, uly], [lrx, lry],[i,j]])
    print("standard",minx, miny, maxx, maxy)
    return matrix


def split(file_name, geotif_crs):
    data_directory = "/home/student/charcoal_hearth_hill/"
    jpgs_output_folder = os.path.join(data_directory,"images/","jpgs/")
    polys_output_folder = os.path.join(data_directory,"polys/")

    #Some areas have both a PAN and a PAS, so we need to distinguish them
    panorpas=""
    if(geotif_crs==32129):
        panorpas="pas"
    if(geotif_crs==32128):
        panorpas="pan"

    if(os.path.exists(file_name)):
        from osgeo import osr
        raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
        print(raw_file_name)
        area_num = raw_file_name[:3]
        if(area_num[-1:] =="b"):
            area_num = area_num[:-1]
        area_num = int(area_num)
        area_str = "0"+str(area_num)
        area_str = area_str[-3:]
        print(area_str)
        driver = gdal.GetDriverByName('GTiff')
        dataset = gdal.Open(file_name)
        proj_dataset = osr.SpatialReference(wkt=dataset.GetProjection())
        print(proj_dataset.GetAttrValue('AUTHORITY',1),geotif_crs)
        
        # Check that the geotif_crs matches what is in the file.
        if(proj_dataset.GetAttrValue('AUTHORITY',1)==str(geotif_crs)):
            print(proj_dataset)
   
            band = dataset.GetRasterBand(1)
            transform = dataset.GetGeoTransform()

            extent = get_extent(dataset)

            cols = int(extent["cols"])
            rows = int(extent["rows"])
            print ("Columns: ", cols)
            print ("Rows: ", rows)

            minx = float(extent["minX"])
            maxx = float(extent["maxX"])
            miny = float(extent["minY"])
            maxy = float(extent["maxY"])
        
            width = int(maxx - minx)
            height = int(maxy - miny)

            nw = int(width/1024)+1
            nh = int(height/768)+1
    
            #******
            # make a polygon for the area and see what points are in the whole area.
            # get the coordinates in lat long

            # Create an empty geopandas GeoDataFrame
            area_hearth_data = gpd.GeoDataFrame()
            area_hearth_data['geometry'] = None
            area_poly = get_poly_with_lat_long_from_geotif(file_name)
            # Read file using gpd.read_file()
            all_hearth_data = gpd.read_file("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp")
            selection = all_hearth_data[0:]
            #print(selection)
            for index, row in selection.iterrows():
                #print("for index, row in selection.iterrows():")
                pt = row['geometry']
                #print(row)
                #select only points inside the bounds of the geotiff AND are confirmed to be hearths
                if(pt.within(area_poly)==True):
                    #print("if(pt.within(tif_poly)==True):")
                    area_hearth_data = area_hearth_data.append(row, ignore_index=True)
    
            if(not area_hearth_data.empty):
            # Determine the output path for the Shapefile
                outfp = os.path.join(data_directory, "area_hearths", "area_" + area_str + panorpas + "_hearth.shp")
                # Write the data into that Shapefile
                area_hearth_data.to_file(outfp)
            #*************************
            #output_path = os.path.join("data", raw_file_name)
            #if not os.path.exists(output_path):
            #    os.makedirs(output_path)

            # Polygons to reflect the tiles
            # Create an empty geopandas GeoDataFrame
            tile_boundary_poly = gpd.GeoDataFrame()
            tile_boundary_poly['geometry'] = None
            tile_boundary_poly.crs = {'init':'epsg:4326'}

            #tiles = create_tiles(minx, miny, maxx, maxy, nw,nh)
            tiles = create_standard_tiles(minx, miny, maxx, maxy)
            transform = dataset.GetGeoTransform()
            xOrigin = transform[0]
            yOrigin = transform[3]
            pixelWidth = transform[1]
            pixelHeight = -transform[5]

            srs = osr.SpatialReference()
            #srs.ImportFromEPSG(32128)
            srs.ImportFromEPSG(geotif_crs)
            #srs.ImportFromEPSG(4326)
            #dst_ds.SetProjection( srs.ExportToWkt() )
    
            tile_num = 0
            for tile in tiles:
                #print (tile)

                minx = tile[0][0]
                maxx = tile[1][0]
                miny = tile[1][1]
                maxy = tile[0][1]
                tilex = "00"+str(tile[2][0])
                tilex = tilex[-2:]
                tiley = "00"+str(tile[2][1])
                tiley = tiley[-2:]
        
                p1 = (minx, maxy)
                p2 = (maxx, miny)

                i1 = int((p1[0] - xOrigin) / pixelWidth)
                j1 = int((yOrigin - p1[1]) / pixelHeight)
                i2 = int((p2[0] - xOrigin) / pixelWidth)
                j2 = int((yOrigin - p2[1]) / pixelHeight)

                new_cols = i2-i1
                new_rows = j2-j1

                data = band.ReadAsArray(i1, j1, new_cols, new_rows)
                #print("data",data.mean(),data)
                #if len(list(set(chain(*data))))<3:
                if(np.all(data==data[0]) or (np.amin(data)==-9999 and np.amax(data)==0)):
                    print("np.all(data==data[0])")
                    #print("data.mean",data.mean())
                else:
                    print("data.mean.too",data.mean(),np.amin(data),np.amax(data) )
                
                    new_x = xOrigin + i1*pixelWidth
                    new_y = yOrigin - j1*pixelHeight

                    #print (new_x, new_y)

                    new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])
                    output_file_name = area_str + panorpas + tiley + tilex
                    output_file_base = output_file_name + ".tif"
                    output_file = os.path.join(data_directory,"images","split_tifs",output_file_base)

                    dst_ds = driver.Create(output_file,
                                           new_cols,
                                           new_rows,
                                           1,
                                           gdal.GDT_Float32)

                    #writing output raster
                    dst_ds.GetRasterBand(1).WriteArray( data )
                    dst_ds.SetProjection( srs.ExportToWkt() )
                    dst_ds.SetGeoTransform(new_transform)
                    tif_metadata = {
                        "minX": str(minx), "maxX": str(maxx),
                        "minY": str(miny), "maxY": str(maxy)
                    }
                    dst_ds.SetMetadata(tif_metadata)
                    dst_ds = None            

                    #make a polygon for this tile
                    tpoly = get_poly_with_lat_long_from_geotif(output_file)
                    new_tp_row = {'id':tile_num, 'geometry':tpoly}
                    tile_boundary_poly = tile_boundary_poly.append(new_tp_row, ignore_index=True)
            
                    # Save .jpg
                    # close tif before re-opening it and saving as jpg
                    # The jpg is to be consumed by the model.
                    
                    tif_to_jpg(output_file,output_file_name,jpgs_output_folder)
                    # /storage/images/charcoal_hearth_hill/images_training/
                    # Are there points inside the geotiff?
                    # annotate_tif_if_it_has_hearths("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp",tpoly,output_file,output_file_name)
                    annotate_tif_if_it_has_hearths(area_hearth_data,tpoly,output_file,output_file_name,geotif_crs)
                
                    #We need to refine to handle points on the border
            
                #Close output raster dataset
                dst_ds = None

                tile_num += 1

            dataset = None
    
            # Determine the output path for the Shapefile
            tile_boundary_poly.crs = ("EPSG:" + str(geotif_crs))
            tile_boundary_poly.geometry = tile_boundary_poly.geometry.to_crs(crs=geotif_crs)
            tile_boundary_poly.to_crs(crs=geotif_crs)
            tile_boundary_poly = tile_boundary_poly.to_crs(epsg=geotif_crs)        
        
            # tile_boundary_poly = tile_boundary_poly.to_crs(epsg=geotif_crs)
    
            outfp = os.path.join(polys_output_folder, (area_str + panorpas + "_tile_boundary.shp"))
            # Write the data into that Shapefile
            tile_boundary_poly.to_file(outfp)
        else:
            print(proj_dataset.GetAttrValue('AUTHORITY',1),geotif_crs," don't match.")
    else:
        print(file_name+" does not exist.")


# Instructions
Download the Slope_DEM.tif files
PAN = 32128
PAS = 32129
Run all cells above and then the one below.

In [78]:
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/41/41blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/43/43blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/46/46blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/52/52partialDEM_Slope.tif",26918)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/55/55blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/58/58blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/12blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/13blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/14blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/24blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/25blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/26blast2demPAN_Slope_DEM.tif",32128) - AttributeError: 'NoneType' object has no attribute 'GetProjection'
#split("/storage/images/28blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/29blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/30blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/31blast2demPAN_Slope_DEM.tif",32128)
#33 has PAN and PAS
#split("/storage/images/part_1/33blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/33blast2demPAS_Slope_DEM.tif",32129)

#split("/storage/images/part_1/34blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/35blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/36blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/37blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/38blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/39blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/40blast2demPAN_Slope_DEM.tif",32128)



#split("/storage/images/part_4/320blast2demPAN_DEM_Slope.tif",32128)

#52partialDEM_Slope.tif",26918)
#58 has PAN and PAS
#60 has PAN and PAS
#83 is missing - now there.
#84 has PAN and PAS
#95 has PAN and PAS
#105 has PAN and PAS
#108 has PAN and PAS
#109 is missing

# on /storage
# 12:29:18 Current Time = 12:30:02 == 44 secs

#on home
# 12:31:29 12:31:51 == 22 secs
        
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
#split("/storage/images/part_1/12blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/52blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/254blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/290blast2demPAS_Slope_DEM.tif",32129)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)



Current Time = 12:58:18
52blast2demPAS_Slope_DEM
052
32129 32129
PROJCS["NAD83 / Pennsylvania South",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["standard_parallel_1",40.96666666666667],
    PARAMETER["standard_parallel_2",39.93333333333333],
    PARAMETER["latitude_of_origin",39.33333333333334],
    PARAMETER["central_meridian",-77.75],
    PARAMETER["false_easting",600000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    AUTHORITY["EPSG","32129"]]
Columns:  10426
Rows

/home/student/anaconda3/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


standard 746761.0658 91440.25844664808 757187.1040537154 101021.2936
11 13
standard 746761.0658 91440.25844664808 757187.1040537154 101021.2936
np.all(data==data[0])
np.all(data==data[0])
data.mean.too -88019.914 -99999.0 18.090195
/home/student/charcoal_hearth_hill/images/split_tifs/052pas1302.tif
pp_minx -75.99172812198677
geotif_crs 32129
                                            geometry    id
0  POLYGON ((-75.99173 40.22708, -75.99173 40.227...  None
epsg:32129
-------
****
src_ds.RasterXSize 1024 365
boundary = poly.bounds
-75.99172812198677
40.22707881118582
-75.99137904158675
40.22742789158582
lon,lat -75.99172812198677 40.22707881118582
p1 -75.99172812198677 40.22707881118582 -748880 100981
lon,lat -75.99137904158675 40.22742789158582
p2 -75.99137904158675 40.22742789158582 -748880 100981
data.mean.too -56598.953 -99999.0 42.153076
/home/student/charcoal_hearth_hill/images/split_tifs/052pas1303.tif
data.mean.too -47852.18 -99999.0 42.031048
/home/student/charcoal_hearth_hill

/home/student/charcoal_hearth_hill/images/split_tifs/052pas1310.tif
np.all(data==data[0])
np.all(data==data[0])
data.mean.too -69605.99 -99999.0 48.6076
/home/student/charcoal_hearth_hill/images/split_tifs/052pas1202.tif
pp_minx -75.99283319957577
pp_minx -75.99127868851754
pp_minx -75.9897506147222
pp_minx -75.99252123987361
pp_minx -75.99061775694518
pp_minx -75.98994096301516
geotif_crs 32129
                                            geometry    id
0  POLYGON ((-75.99283 40.22245, -75.99283 40.222...  None
1  POLYGON ((-75.99128 40.22242, -75.99128 40.222...  None
2  POLYGON ((-75.98975 40.22352, -75.98975 40.223...  None
3  POLYGON ((-75.99252 40.22074, -75.99252 40.221...  None
4  POLYGON ((-75.99062 40.22577, -75.99062 40.226...  None
5  POLYGON ((-75.98994 40.22022, -75.98994 40.220...  None
epsg:32129
-------
****
src_ds.RasterXSize 1024 768
boundary = poly.bounds
-75.99283319957577
40.222447002726604
-75.99248411917576
40.2227960831266
lon,lat -75.99283319957577 40.222447002

pp_minx -75.93507306760257
pp_minx -75.93445443565082
pp_minx -75.93315900976896
pp_minx -75.93275187592037
pp_minx -75.932291867546
geotif_crs 32129
                                            geometry    id
0  POLYGON ((-75.93542 40.22007, -75.93542 40.220...  None
1  POLYGON ((-75.93507 40.22102, -75.93507 40.221...  None
2  POLYGON ((-75.93445 40.22174, -75.93445 40.222...  None
3  POLYGON ((-75.93316 40.22116, -75.93316 40.221...  None
4  POLYGON ((-75.93275 40.22046, -75.93275 40.220...  None
5  POLYGON ((-75.93229 40.22096, -75.93229 40.221...  None
epsg:32129
-------
****
src_ds.RasterXSize 1024 768
boundary = poly.bounds
-75.9354167520202
40.22007293651865
-75.93506767162019
40.22042201691865
lon,lat -75.9354167520202 40.22007293651865
p1 -75.9354167520202 40.22007293651865 -754000 100616
lon,lat -75.93506767162019 40.22042201691865
p2 -75.93506767162019 40.22042201691865 -754000 100616
boundary = poly.bounds
-75.93507306760257
40.221019390530294
-75.93472398720256
40.22136847

...

/home/student/charcoal_hearth_hill/images/split_tifs/052pas0505.tif
data.mean.too 4.6324906 0.0 32.885292
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0506.tif
data.mean.too 4.347519 0.0 89.4727
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0507.tif
data.mean.too -4270.5327 -99999.0 89.54354
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0508.tif
data.mean.too -63931.562 -99999.0 70.868576
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0509.tif
data.mean.too -66799.26 -99999.0 39.89721
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0510.tif
data.mean.too -93.09832 -99999.0 89.29191
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0400.tif
data.mean.too 5.2593846 0.0 56.431126
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0401.tif
data.mean.too 6.232769 0.0 39.274784
/home/student/charcoal_hearth_hill/images/split_tifs/052pas0402.tif
data.mean.too 6.371428 0.0 50.593586
/home/student/charcoal_hearth_hi

/home/student/anaconda3/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


standard 661417.0 121920.0 670562.0 134113.0
9 16
standard 661417.0 121920.0 670562.0 134113.0
data.mean.too -6.4252887 -9999.0 89.83663
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1600.tif
data.mean.too 4.736071 -9999.0 89.8082
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1601.tif
data.mean.too 3.482407 -9999.0 89.81121
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1602.tif
data.mean.too -4.0001745 -9999.0 89.76543
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1603.tif
data.mean.too -12.319829 -9999.0 89.72834
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1604.tif
data.mean.too -0.9708448 -9999.0 89.75396
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1605.tif
np.all(data==data[0])
np.all(data==data[0])
np.all(data==data[0])
data.mean.too 13.010999 -9999.0 89.804054
/home/student/charcoal_hearth_hill/images/split_tifs/254pas1500.tif
data.mean.too 22.267021 0.0 81.59039
/home/student/charcoal_hearth_hill/im

/home/student/charcoal_hearth_hill/images/split_tifs/254pas0704.tif
data.mean.too 22.12702 0.0 75.00565
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0705.tif
data.mean.too 16.270847 0.0 89.718414
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0706.tif
data.mean.too 7.3286614 0.0 89.718285
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0707.tif
data.mean.too 12.906733 -9999.0 89.795525
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0708.tif
data.mean.too 23.413857 -9999.0 89.81386
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0600.tif
data.mean.too 31.692804 0.0 83.114685
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0601.tif
data.mean.too 21.946335 0.0 73.140465
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0602.tif
data.mean.too 27.83467 0.0 77.26303
/home/student/charcoal_hearth_hill/images/split_tifs/254pas0603.tif
data.mean.too 26.188675 0.0 85.16436
/home/student/charcoal_hearth_hill/images/sp

/home/student/charcoal_hearth_hill/images/split_tifs/290pas0607.tif
data.mean.too 9.428784 -9999.0 89.82089
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0608.tif
np.all(data==data[0])
np.all(data==data[0])
data.mean.too 0.9476557 0.0 89.7915
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0502.tif
data.mean.too 18.31599 0.0 89.705696
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0503.tif
data.mean.too 18.691301 0.0 89.70674
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0504.tif
data.mean.too 12.904799 0.0 89.711655
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0505.tif
data.mean.too 5.612329 0.0 89.71047
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0506.tif
data.mean.too 31.548025 0.0 78.72065
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0507.tif
data.mean.too 10.610525 -9999.0 89.79569
/home/student/charcoal_hearth_hill/images/split_tifs/290pas0508.tif
data.mean.too 16.595922 -9999.0 89.81872
/

In [ ]:
# QGIS only
# This is just here to store and keep track of.
# In QGIS due to loading a large number of files, it gets unstable.
# This program loads all of the area polys to make sure processing worked correctly.
# It will print which layers did not load.
# qgis_1_check_predictions
from qgis.core import *
import qgis.utils

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"
#/storage/images/charcoal_hearth_hill/polys/
# The format is:
# vlayer = QgsVectorLayer(data_source, layer_name, provider_name)

   
vlayer = iface.addVectorLayer(path_to_layer, "Ports layer", "ogr")
if not vlayer:
    print("Layer failed to load!")

report=""
import csv
with open('/home/student/charcoalhearths/data_sheet.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Area'])
        area=row['Area']
        area="0"+area
        area=area[-3:]
        pan=""
        pas=""
        pan = row['PAN']
        if(pan == "Y"):
            path_to_layer = "/home/student/charcoal_hearth_hill/polys/cfg20200720T1614/"+area+"pan_predictions.shp"
            vlayer = iface.addVectorLayer(path_to_layer, area+"pan layer", "ogr")
            if not vlayer:
                print(area+ "pan layer failed to load!")
                report = report + "\n" + area+ "pan_predictions layer failed to load!"
        pas = row['PAS']
        if(pas == "Y"):
            path_to_layer = "/home/student/charcoal_hearth_hill/polys/cfg20200720T1614/"+area+"pan_predictions.shp"
            vlayer = iface.addVectorLayer(path_to_layer, area+"pas layer", "ogr")
            if not vlayer:
                print(area+ "pas_predictions layer failed to load!")
                report = report + "\n" + area+ "pas layer failed to load!

print(report)  


In [ ]:
# qgis_2_add_poly_layers_for_nort_east
# This program loads all of the area polys to make sure processing worked correctly.
# It will print which layers did not load.
from qgis.core import *
import qgis.utils

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"
   
vlayer = iface.addVectorLayer(path_to_layer, "Charcoal Hearths layer", "ogr")
if not vlayer:
    print("Layer failed to load!")

report=""
import csv

areas_nsew = {}
with open('/home/student/charcoalhearths/data_sheet_areas_nsew.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Number'],row['NSEW'])
        area = row['Number']
        area="0"+area
        area=area[-3:]
        nsew = row['NSEW']
        areas_nsew[area] = nsew

active_region = "NE"

#layer style
panSymbol = QgsFillSymbolV2.createSimple({'color':'0,0,0,0', 'color_border':'#00ff00', 'width_border':'0.3'})
pasSymbol = QgsFillSymbolV2.createSimple({'color':'0,0,0,0', 'color_border':'#0000ff', 'width_border':'0.3'})

with open('/home/student/charcoalhearths/data_sheet.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Area'])
        area=row['Area']
        area="0"+area
        area=area[-3:]
        if(areas_nsew[area] == active_region):
            pan=""
            pas=""
            pan = row['PAN']
            if(pan == "Y"):
                path_to_layer = "/home/student/charcoal_hearth_hill/polys/"+area+"pan_tile_boundary.shp"
                vlayer = iface.addVectorLayer(path_to_layer, area+"pan layer", "ogr")
                if not vlayer:
                    print(area+ "pan layer failed to load!")
                    report = report + "\n" + area+ "pan layer failed to load!"
                else:
                    layerRenderer  = vlayer.rendererV2()
                    layerRenderer.setSymbol(panSymbol)
                    vlayer.triggerRepaint()
                #Predictions
                path_to_layer = "/home/student/charcoal_hearth_hill/polys/cfg20200720T1614/"+area+"pan_predictions.shp"
                vlayer = iface.addVectorLayer(path_to_layer, area+"pan layer", "ogr")
                if not vlayer:
                    print(area+ "pan_predictions layer failed to load!")
                    report = report + "\n" + area+ "pan_predictions layer failed to load!"


            pas = row['PAS']
            if(pas == "Y"):
                path_to_layer = "/home/student/charcoal_hearth_hill/polys/"+area+"pas_tile_boundary.shp"
                vlayer = iface.addVectorLayer(path_to_layer, area+"pas layer", "ogr")
                if not vlayer:
                    print(area+ "pas layer failed to load!")
                    report = report + "\n" + area+ "pas layer failed to load!"
                else:
                    layerRenderer  = vlayer.rendererV2()
                    layerRenderer.setSymbol(pasSymbol)
                    vlayer.triggerRepaint()
                #Predictions
                path_to_layer = "/home/student/charcoal_hearth_hill/polys/cfg20200720T1614/"+area+"pas_predictions.shp"
                vlayer = iface.addVectorLayer(path_to_layer, area+"pas layer", "ogr")
                if not vlayer:
                    print(area+ "pas_predictions layer failed to load!")
                    report = report + "\n" + area+ "pas_predictions layer failed to load!"                    

            #vlayer.setCustomProperty("labeling", "pal")
            #vlayer.setCustomProperty("labeling/enabled", "true")
            #vlayer.setCustomProperty("labeling/fontFamily", "Arial")
            #vlayer.setCustomProperty("labeling/fontSize", "10")
            #vlayer.setCustomProperty("labeling/fieldName", "layerName")
            #vlayer.setCustomProperty("labeling/placement", "2")
print(report) 

In [ ]:
# qgis_3_load_rasters_north_east
# This program loads the rasters in the NE of PA to make sure processing worked correctly.
# It will print which layers did not load.
from qgis.core import *
import qgis.utils

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"
  
vlayer = iface.addVectorLayer(path_to_layer, "Charcoal Hearths layer", "ogr")
if not vlayer:
    print("Layer failed to load!")

# load the predictions
path_to_layer = "/home/student/charcoal_hearth_hill/polys/cfg20200720T1614/4326_000_hearth_prediction_points.shp"
  
vlayer = iface.addVectorLayer(path_to_layer, "Charcoal Hearths predictions layer", "ogr")
if not vlayer:
    print("Layer failed to load!")

report=""
import csv

areas_nsew = {}
with open('/home/student/charcoalhearths/data_sheet_areas_nsew.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Number'],row['NSEW'])
        area = row['Number']
        area="0"+area
        area=area[-3:]
        nsew = row['NSEW']
        areas_nsew[area] = nsew

active_region = "NE"

#layer style
panSymbol = QgsFillSymbolV2.createSimple({'color':'0,0,0,0', 'color_border':'#00ff00', 'width_border':'0.3'})
pasSymbol = QgsFillSymbolV2.createSimple({'color':'0,0,0,0', 'color_border':'#0000ff', 'width_border':'0.3'})

with open('/home/student/charcoalhearths/data_sheet.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Area'])
        area=row['Area']
        area="0"+area
        area=area[-3:]
        if(areas_nsew[area] == active_region):
            pan=""
            pas=""
            pan = row['PAN']
            
            area_num = int(area)
            part_num = ""
            if(area_num > 0 and area_num < 98):
                part_num = "1"
            if(area_num > 97 and area_num < 174):
                part_num = "2"
            if(area_num > 173 and area_num < 252):
                part_num = "3"
            if(area_num > 251 and area_num < 336):
                part_num = "4"
                
            if(pan == "Y"):
                path_to_layer = "/storage/images/part_" + part_num + "/" + str(area_num) + "blast2demPAN_Slope_DEM.tif"
                
                vlayer = iface.addRasterLayer(path_to_layer, area+"pan layer")
                if not vlayer:
                    print(area+ "pan layer failed to load!")
                    report = report + "\n" + area+ "pan layer failed to load!"

            pas = row['PAS']
            if(pas == "Y"):
                path_to_layer = "/storage/images/part_" + part_num + "/" + str(area_num) + "blast2demPAS_Slope_DEM.tif"
                vlayer = iface.addVectorLayer(path_to_layer, area+"pas layer", "ogr")
                if not vlayer:
                    print(area+ "pas layer failed to load!")
                    report = report + "\n" + area+ "pas layer failed to load!"
print(report) 